<a href="https://colab.research.google.com/github/theresaltrinh/project_voting/blob/main/voting_model_building_tori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voting Project -- Model Building

In [1]:
! git clone https://github.com/theresaltrinh/project_voting

Cloning into 'project_voting'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 85 (delta 12), reused 6 (delta 3), pack-reused 63
Receiving objects: 100% (85/85), 31.52 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Updating files: 100% (45/45), done.


In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

vadf = pd.read_csv('/content/drive/My Drive/4th Year/IntroML/DS3001 Projects/Voting Project/cleaned_voting_VA.csv', index_col=0)
cdf = pd.read_csv('/content/drive/My Drive/4th Year/IntroML/DS3001 Projects/Voting Project/cleaned_county_adjacencies.csv', index_col=0)

Mounted at /content/drive


In [26]:
vadf.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,winner
0,2000,Accomack,51001,Al Gore,Democrat,5092,11925,None
1,2000,Accomack,51001,George W. Bush,Republican,6352,11925,Republican
2,2000,Accomack,51001,Ralph Nader,Green,220,11925,None
3,2000,Accomack,51001,Other,Other,261,11925,None
4,2000,Albemarle,51003,Al Gore,Democrat,16255,36846,None


In [21]:
vadf['winner'] = vadf.groupby(['year', 'countyCode'])['candidatevotes'].transform(max) == vadf['candidatevotes']

In [23]:
vadf.loc[vadf['winner'], 'winner'] = vadf['party']
vadf['winner'] = vadf['winner'].replace({False: None})

# To only have one row per county per year, we might want to remove the rows that didn't win
# Keeping only the rows with winners
vadf_winner = vadf.dropna(subset=['winner'])

In [33]:
vadf_winner.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,winner
1,2000,Accomack,51001,George W. Bush,Republican,6352,11925,Republican
5,2000,Albemarle,51003,George W. Bush,Republican,18291,36846,Republican
9,2000,Alleghany,51005,George W. Bush,Republican,2808,5123,Republican
13,2000,Amelia,51007,George W. Bush,Republican,2947,4788,Republican
17,2000,Amherst,51009,George W. Bush,Republican,6660,11712,Republican


In [41]:
total_votes_by_year_party = vadf.groupby(['year', 'party'])['candidatevotes', ].sum().reset_index()

most_votes_by_year = total_votes_by_year_party.loc[total_votes_by_year_party.groupby('year')['candidatevotes'].idxmax()]
most_votes_by_year
# Who actually won each year based on number of votes

,year,party,candidatevotes
3,2000,Republican,1436877
6,2004,Republican,1716959
7,2008,Democrat,1959532
10,2012,Democrat,1971820
13,2016,Democrat,1981473
16,2020,Democrat,2413568


In [36]:
# Now we move on trying to predict for 2024

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

party_votes = vadf.groupby(['year', 'party'])['candidatevotes'].sum().reset_index()
democratic_votes = party_votes[party_votes['party'] == 'Democrat']
X_dem = democratic_votes[['year']]
y_dem = democratic_votes['candidatevotes']
X_train_dem, X_test_dem, y_train_dem, y_test_dem = train_test_split(X_dem, y_dem, test_size=0.2, random_state=42)

model_dem = LinearRegression()
model_dem.fit(X_train_dem, y_train_dem)
y_pred_dem = model_dem.predict(X_test_dem)

In [43]:
rmse_dem = mean_squared_error(y_test_dem, y_pred_dem, squared=False)
print(f'RMSE: {rmse_dem}')

# Predict the votes for 2024
predicted_votes_2024_dem = model_dem.predict([[2024]])
print(f'Predicted votes for the Democratic Party in 2024: {predicted_votes_2024_dem[0]}')

RMSE: 338289.38916555094
Predicted votes for the Democratic Party in 2024: 2424538.5


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [44]:
republican_votes = party_votes[party_votes['party'] == 'Republican']
X_rep = republican_votes[['year']]
y_rep = republican_votes['candidatevotes']
X_train_rep, X_test_rep, y_train_rep, y_test_rep = train_test_split(X_rep, y_rep, test_size=0.2, random_state=42)

model_rep = LinearRegression()
model_rep.fit(X_train_rep, y_train_rep)
y_pred_rep = model_rep.predict(X_test_rep)

In [46]:
rmse_rep = mean_squared_error(y_test_rep, y_pred_rep, squared=False)
print(f'RMSE: {rmse_rep}')

# Predict the votes for 2024
predicted_votes_2024_rep = model_rep.predict([[2024]])
print(f'Predicted votes for the Republican Party in 2024: {predicted_votes_2024_rep[0]}')

RMSE: 116219.62567346507
Predicted votes for the Republican Party in 2024: 1984649.0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [49]:
if predicted_votes_2024_rep > predicted_votes_2024_dem:
  print("Republican win")
else:
  print("Democrat win")

Democrat win
